In [ ]:
import os
import simplejson as json

class Prepare:
    def __init__(self):
        self.json_name = '1.json'
        with open(self.json_name) as data_file:    
            self.json = json.load(data_file)
        self.path = self.json['path0']
    
    # add all scenes from landsats_done
    def add_scenes_to_json(self):
        dirs = [d for d in os.listdir(self.path) if os.path.isdir(os.path.join(self.path, d))]
        self.json['scenes'] = []
        id = 0
        for d in dirs:
            tmp = {}
            tmp['id'] = id
            id += 1
            tmp['path'] = d
            tmp['bit'] = '16' if (d[2] == '8') else '8'
            self.json['scenes'].append(tmp)
        with open(self.json_name, 'w') as outfile:
            json.dump(self.json, outfile, sort_keys = True, indent = 4, ensure_ascii = False)

    # copy all need files from landsats_done
    def copy_need_files(self):
        path_in = self.json['path']
        dirs = [os.path.join(path_in, i) for i in 
                ['in', 'in_filtered', 'in/red', 'in/nir', 'in_filtered/red', 'in_filtered/nir']]
        try:
            for i in dirs:
                os.mkdir(i)
        except:
            raise Exception('copy_need_files: dirs can not be created or exist')
        for i in self.json['scenes']:
            if (i['bit'] == '8'):
                suffs = ['_B3_C.tif', '_B4_C.tif', '_B3_C_F.tif', '_B4_C_F.tif']
            else:
                suffs = ['_B4_C.tif', '_B5_C.tif', '_B4_C_F.tif', '_B5_C_F.tif']
            for j in range(len(suffs)):
                os.system("cp " + os.path.join(self.path, i['path'], i['path'] + suffs[j]) + " " + 
                          os.path.join(dirs[j + 2], i['path'] + '.tif'))
    
pr = Prepare()
#pr.add_scenes_to_json()
pr.copy_need_files()

In [4]:
import os
import simplejson as json
import numpy as np
import skimage.io
import matplotlib
import scipy.spatial.distance

# class for one scene analyse
class Scene:
    def __init__(self, _id_):
        self.id = _id_
        with open('1.json') as data_file:    
            self.json = json.load(data_file)
        self.bit = self.json['scenes'][self.id]['bit']
        self.path = self.json['scenes'][self.id]['path']
            
    def is_in_mask(self, img, x, y):
        if self.bit == '16':
            return img[y, x] < 64000 # strange constant
        else:
            return img[y, x] < 253
        
    def get_white_pixel(self):
        if self.bit == '16':
            return 65535
        else:
            return 255

    def read_images(self, path):
        return {'red': skimage.io.imread(os.path.join(self.json['path'], self.json['paths'][path], 'red',
                                                     self.path +  self.json['suffs'][path])),
                'nir': skimage.io.imread(os.path.join(self.json['path'], self.json['paths'][path], 'nir',
                                                     self.path +  self.json['suffs'][path]))}
    
    def get_plot_array(self, img_red, img_nir):
        sizes = self.json['plotSizes'][self.bit]
        plot_array = np.zeros([sizes['red'][1] - sizes['red'][0] + 1,
                               sizes['nir'][1] - sizes['nir'][0] + 1], dtype = 'int32')
        for y, x in np.ndindex(img_red.shape):
            if self.is_in_mask(img_red, x, y) and self.is_in_mask(img_nir, x, y) \
                and sizes['red'][0] <= img_red[y, x] <= sizes['red'][1] \
                and sizes['nir'][0] <= img_nir[y, x] <= sizes['nir'][1]:
                    plot_array[img_nir[y, x] - sizes['nir'][0], img_red[y, x] - sizes['red'][0]] += 1
        return plot_array
    
    def save_plot_array(self, plot_array):
        matplotlib.image.imsave(os.path.join(self.json['path'], self.json['paths']['notMarkedPlots'], 
                                             self.path + self.json['suffs']['markedPlots']), 
                                plot_array[::-1, :])
        
    def make_solp_scene(self):
        imgs = self.read_images('inFiltered')
        sizes = self.json['plotSizes'][self.bit]
        plot_marked = skimage.io.imread(os.path.join(self.json['path'], self.json['paths']['markedPlots'], 
                                                     self.path + self.json['suffs']['markedPlots']))
        colors = self.json['colorsMarkedPlots'][1:2:1]
        print colors
        color_eps = 9
        for x, y in np.ndindex(imgs['red'].shape):
            red, nir = imgs['red'][x, y], imgs['nir'][x, y]
            if not sizes['red'][0] <= red <= sizes['red'][1] or not sizes['nir'][0] <= nir <= sizes['nir'][1]:
                for img in imgs.values():
                    img[x, y] = self.get_white_pixel()
                continue
            for color in colors:
                if scipy.spatial.distance.pdist([plot_marked[sizes['nir'][1] - nir, red - sizes['red'][0]][0:3:1],
                                                 color], 'minkowski', 1)[0] < color_eps:
                    break
            else:
                for img in imgs.values():
                    img[x, y] = self.get_white_pixel()
        ch_arr = ['red', 'nir']
        for ch in ch_arr:
            skimage.io.imsave(os.path.join(self.json['path'], self.json['paths']['solp'], ch, 
                                           self.path + self.json['suffs']['solp']), 
                              imgs[ch], plugin = 'tifffile')
    
    def get_soil_line(self, imgs):
        plot_array = A.get_plot_array(imgs['red'], imgs['nir'])
        red, nir = [], []
        for x, y in np.ndindex(imgs['red'].shape):
            red.append(imgs['red'][x, y])
            nir.append(imgs['nir'][x, y])
        red_matrix = np.vstack([red, np.ones(len(red))]).T
        return np.linalg.lstsq(red_matrix, nir)[0]
    
    def brat_vec(self, imgs, pnt, rad, line):
        res_x, res_y, ave_x, ave_y, ave_subj, n = 0, 0, 0, 0, 0, 0
        for y in np.arange(max(pnt['y'] - rad, 0), min(pnt['y'] + rad, imgs['red'].shape[0])):
            for x in np.arange(max(pnt['x'] - rad, 0), min(pnt['x'] + rad, imgs['red'].shape[1])):
                if (pnt['x'] - x) ** 2 + (pnt['y'] - y) ** 2 <= rad ** 2 \
                    and self.is_in_mask(imgs['red'], x, y) and self.is_in_mask(imgs['nir'], x, y):
                        n += 1
                        subj = float(-line[0]) * imgs['red'][y, x] - line[1] + imgs['nir'][y, x]
                        res_x += x * subj
                        res_y += y * subj
                        ave_x += x
                        ave_y += y
                        ave_subj += subj
        return {'xx': (res_x - ave_x * ave_subj) / float(n) if n > 0 else -9999,
               'yy': (res_y - ave_y * ave_subj) / float(n) if n > 0 else -9999,
               'avg': ave_subj / float(n) if n > 0 else -9999,
               'n': n}
    
    def brat_f1(self):
        json_f1 = self.json['brat_f1']
        rad, x_pts, y_pts = json_f1['rad'], json_f1['xPts'], json_f1['yPts']
        imgs = self.read_images('in')
        plot_array = self.get_plot_array(imgs['red'], imgs['nir'])
        line = self.get_soil_line(imgs)
        x_abs_0, y_abs_0 = json_f1['xAbs0'], json_f1['yAbs0']
        x_abs_step, y_abs_step = json_f1['xAbsStep'], json_f1['yAbsStep']
        need_comma = 0
        
        f = open(self.json['paths']['bratTxt'] + self.path + self.json['suffs']['bratTxt'], 'w')
        for i in range(x_pts):
            for j in range(y_pts):
                print i, j, "\n"
                x = float(i) / (x_pts - 1) * imgs['red'].shape[1]
                y = float(j) / (y_pts - 1) * imgs['red'].shape[0]
                x_abs = x_abs_0 + x * x_abs_step
                y_abs = y_abs_0 + y * y_abs_step
                vec = self.brat_vec(imgs, {'x': x, 'y': y}, rad, line)
                if need_comma:
                    f.write(',')
                f.write('{{%i,%i},{%i,%i},{%f,%f},%f,%i}\n' % 
                        (x, y, x_abs, y_abs, float(vec['xx']), float(vec['yy']), float(vec['avg']), vec['n']))
                if not need_comma:
                    need_comma = 1
        f.write('}')
        close(f)

In [5]:
A = Scene(5)
#A.brat_f1()
#A.make_solp_scene()
#A.get_soil_line()
imgs = A.read_images('in')
%time plot_array = A.get_plot_array(imgs['red'], imgs['nir'])
#A.save_plot_array(plot_array)

CPU times: user 1min 12s, sys: 278 ms, total: 1min 12s
Wall time: 1min 11s


In [ ]:
%pylab inline
import matplotlib.pyplot as plt
plt.imshow(plot_array[::-1, :])
#plt.xlim([0, 255])
#plt.ylim([255, 0])
#plt.axis('off')
#ax = plt.gca()
#ax.xaxis.set_major_formatter(plt.NullFormatter())
#ax.yaxis.set_major_formatter(plt.NullFormatter())
c = plt.gcf()
#plt.gcf().set_size_inches(1, 1)
#c.subplots_adjust(bottom=0.,left=0.,right=1.,top=1.)


#plt.autoscale(True, 'both')

plt.axes().axes.get_yaxis().set_visible(False)
plt.axes().axes.get_xaxis().set_visible(False)


c = plt.gcf()
c.set_size_inches(1., 1.)
#c.set_dpi(256 / 2.00)

plt.savefig('test2.png', dpi=256, bbox_inches = 'tight', pad_inches = 0)

plt.show()

In [ ]:
import matplotlib
matplotlib.image.imsave('name.png', plot_array[::-1, :])

In [ ]:
str = "/media/fila/DATA/landsat/landsats_done"
str1 = "/media/fila/DATA/landsat/in"
with open('1.json') as data_file:    
    data = json.load(data_file)
for i in data['scenes']:
    file1 = os.path.join(str, i['path'], i['path'] + i['suffNir'])
    file2 = os.path.join(str, i['path'], i['path'] + i['suffRed'])
    os.system("cp " + file1 + " " + str1)
    os.system("cp " + file2 + " " + str1)

In [ ]:
%time img = skimage.io.imread('/media/fila/DATA/landsat/in/red/LE71780232001124SGS01.tif')
print img.shape

In [ ]:
qqq = [1,2,3,4,5,6,7,8]
qqq[1:4]

In [ ]:
for x, y in np.ndindex(img.shape[0:2:1]):
    p = img[x, y]
    if abs(p[0] - 100) <= 5 and abs(p[1] - 255) <= 5 and abs(p[2] - 100) <= 5:
        print p

In [ ]:
for i in range(5):
    for j in range(5):
        break
    else:
        print '111'

In [ ]:
a = {'red': 123, 'nir': 234}
for aaa in a.values():
    print aaa

In [ ]:
qqq = np.array([1,2])
print qqq[1:2:1]

In [ ]:
min(0, -1.00)

In [2]:
%load_ext Cython
import skimage.io
img = skimage.io.imread('/media/fila/DATA/landsat/in/red/LE71780232001124SGS01.tif')


In [3]:
%%cython
cimport cython
cimport numpy as np

@cython.boundscheck(False)
def ff(np.ndarray img1):
    b = '12312312s'
    b = 1
    cdef int s = 0
    cdef unsigned int x = 0
    cdef unsigned int y = 0
    for x in range(img1.shape[1]):
        for y in range(img1.shape[0]):
            s += <unsigned int>img1[<unsigned int>y, <unsigned int>x]
    print s

In [23]:
%time ff(img)

305767252
CPU times: user 530 ms, sys: 0 ns, total: 530 ms
Wall time: 529 ms


In [10]:
import numpy as np

def fff(img1):
    s = 0
    for x in range(img1.shape[1]):
        for y in range(img1.shape[0]):
            s += img1[y, x]
    print s

In [11]:
%time fff(img)

305767252
CPU times: user 1.27 s, sys: 20.1 ms, total: 1.29 s
Wall time: 1.26 s


In [ ]:
def gg():
    cdef int s
    s = 0
    while (s < 10**8):
        s += 1
gg()

In [9]:
def get_soil_line(self, imgs):
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        cdef list red = [], nir = []
        cdef int x, y
        for x in range(img_red.shape[1]):
            for y in range(img_red.shape[0]):
                red.append(img_red[y, x])
                nir.append(img_nir[y, x])
        #return np.linalg.lstsq(np.vstack([red, np.ones(len(red))]).T, nir)[0]

(2181, 2882)

In [ ]:
def get_soil_line(self, imgs):
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        #cdef list red = [], nir = []
        cdef np.ndarray[np.uint16_t, ndim = 1] red = np.zeros([img_red.size], dtype = 'uint16')
        cdef np.ndarray[np.uint16_t, ndim = 1] nir = np.zeros([img_red.size], dtype = 'uint16')
        cdef int x, y, size = 0
        for x in range(img_red.shape[1]):
            for y in range(img_red.shape[0]):
                red[size] = imgs['red'][y, x]
                nir[size] = imgs['nir'][y, x]
        #return np.linalg.lstsq(np.vstack([red, np.ones(len(red))]).T, nir)[0]